In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import os 
from multiprocessing import Pool

# ========================== DIFUSIÓN VECTORIAL =============================

def difusion_skin(m, n, t, dt, nu, u_init, Gamma):
    u_prev = np.zeros([m, n])
    u_curr = np.zeros([m, n])
    Gamma = Gamma * nu * dt

    core_slice = (slice(1, m-1), slice(1, n-1))
    gamma_slice = (slice(1, m-1), slice(1, n-1), slice(None))

    neighbor_indices = [
        (slice(2, m), slice(1, n-1)),     # East
        (slice(2, m), slice(2, n)),       # NE
        (slice(1, m-1), slice(2, n)),     # N
        (slice(0, m-2), slice(2, n)),     # NW
        (slice(0, m-2), slice(1, n-1)),   # West
        (slice(0, m-2), slice(0, n-2)),   # SW
        (slice(1, m-1), slice(0, n-2)),   # S
        (slice(2, m), slice(0, n-2))      # SE
    ]

    boundary_indices = {
        'inlet': (slice(None), 0),
        'outlet': (slice(None), n-1),
        'outlet_ref': (slice(None), n-2),
        'left': (m-1, slice(None)),
        'left_ref': (m-2, slice(None)),
        'right': (0, slice(None)),
        'right_ref': (1, slice(None))
    }

    neighbor_arrays = np.zeros((m-2, n-2, 9))

    for k in range(1, t):
        neighbor_arrays[:, :, 0] = u_prev[core_slice]
        for i in range(8):
            neighbor_arrays[:, :, i+1] = u_prev[neighbor_indices[i]]

        u_curr[core_slice] = u_prev[core_slice] + np.einsum('ijk,ijk->ij', Gamma[gamma_slice], neighbor_arrays)

        u_curr[boundary_indices['inlet']] = u_init
        u_curr[boundary_indices['outlet']] = u_curr[boundary_indices['outlet_ref']]
        u_curr[boundary_indices['left']] = u_curr[boundary_indices['left_ref']]
        u_curr[boundary_indices['right']] = u_curr[boundary_indices['right_ref']]

        u_prev, u_curr = u_curr, u_prev

    return u_prev

# ========================== GAMMAS (COEFICIENTES GFD) ======================

def Gammas(x, y, L):
    m, n = x.shape
    Gamma = np.zeros([m, n, 9])

    i_range = slice(1, m-1)
    j_range = slice(1, n-1)
    x_center = x[i_range, j_range]
    y_center = y[i_range, j_range]

    dx_neighbors = np.stack([
        x[2:m, 1:n-1] - x_center,
        x[2:m, 2:n] - x_center,
        x[1:m-1, 2:n] - x_center,
        x[0:m-2, 2:n] - x_center,
        x[0:m-2, 1:n-1] - x_center,
        x[0:m-2, 0:n-2] - x_center,
        x[1:m-1, 0:n-2] - x_center,
        x[2:m, 0:n-2] - x_center
    ], axis=-1)

    dy_neighbors = np.stack([
        y[2:m, 1:n-1] - y_center,
        y[2:m, 2:n] - y_center,
        y[1:m-1, 2:n] - y_center,
        y[0:m-2, 2:n] - y_center,
        y[0:m-2, 1:n-1] - y_center,
        y[0:m-2, 0:n-2] - y_center,
        y[1:m-1, 0:n-2] - y_center,
        y[2:m, 0:n-2] - y_center
    ], axis=-1)

    M_matrices = np.stack([
        dx_neighbors,
        dy_neighbors,
        dx_neighbors**2,
        dx_neighbors * dy_neighbors,
        dy_neighbors**2
    ], axis=2)

    original_shape = M_matrices.shape[:2]
    M_reshaped = M_matrices.reshape(-1, 5, 8)

    M_pinv_list = [np.linalg.pinv(M_reshaped[k]) for k in range(M_reshaped.shape[0])]
    M_pinv = np.array(M_pinv_list).reshape(original_shape[0], original_shape[1], 8, 5)

    L_flat = L.flatten()
    YY = np.einsum('ijkl,l->ijk', M_pinv, L_flat)
    Gamma_0 = -np.sum(YY, axis=2)

    Gamma[1:m-1, 1:n-1, 0] = Gamma_0
    Gamma[1:m-1, 1:n-1, 1:9] = YY

    return Gamma

# ========================== MAIN ===========================================

# Cargar datos de la malla
datos = scipy.io.loadmat('piel224.mat')
x, y = datos["x"], datos["y"]
m, n = x.shape
output_dir = 'skin224'

# Operador Laplaciano
L = np.vstack([[0], [0], [2], [0], [2]])
Gamma = Gammas(x, y, L)

# ========================== FUNCION PARA PARALELIZAR =======================

def run_case(params):
    ss, j = params
    nu = ss * 1e-9
    u_init = j

    # Definir parámetros locales
    t_local = 500
    T_local = np.linspace(0, 20 * 3600, t_local)
    dt_local = T_local[1] - T_local[0]

    dx_min = np.min(np.sqrt((x[1:, :] - x[:-1, :])**2 + (y[1:, :] - y[:-1, :])**2))
    alpha = nu * dt_local / dx_min**2

    while alpha > 0.05:
        t_local = int(t_local * 1.1)
        T_local = np.linspace(0, 20 * 3600, t_local)
        dt_local = T_local[1] - T_local[0]
        alpha = nu * dt_local / dx_min**2

    print(f"Simulando para Ci={j}, nu={nu:.2e}...")

    u_ap = difusion_skin(m, n, t_local, dt_local, nu, u_init, Gamma)
    if u_ap is None:
        return

    carpeta_nu = os.path.join(output_dir, f"nu_{nu:.9f}")
    os.makedirs(carpeta_nu, exist_ok=True)

    plt.figure(figsize=(8, 6))
    plt.pcolormesh(x, y, u_ap, shading='auto', cmap='cool_r', vmin=0, vmax=900)
    plt.axis('off')

    filename = os.path.join(carpeta_nu, f"c_{j:03d}.png")
    plt.savefig(filename, dpi=75, bbox_inches='tight', pad_inches=0)
    plt.close()

    print(f"Guardado: {filename}")

# ========================== EJECUCIÓN ======================================

if __name__ == "__main__":
    combinations = [(i, j) for i in range(1, 201) for j in range(1, 901)]
    with Pool(processes=1) as pool:
        pool.map(run_case, combinations)

Simulando para Ci=1, nu=1.00e-09...


/tmp/ipykernel_42741/756432562.py:155: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  plt.pcolormesh(x, y, u_ap, shading='auto', cmap='cool_r', vmin=0, vmax=900)


Guardado: skin224/nu_0.000000001/c_001.png
Simulando para Ci=2, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_002.png
Simulando para Ci=3, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_003.png
Simulando para Ci=4, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_004.png
Simulando para Ci=5, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_005.png
Simulando para Ci=6, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_006.png
Simulando para Ci=7, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_007.png
Simulando para Ci=8, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_008.png
Simulando para Ci=9, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_009.png
Simulando para Ci=10, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_010.png
Simulando para Ci=11, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_011.png
Simulando para Ci=12, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_012.png
Simulando para Ci=13, nu=1.00e-09...
Guardado: skin224/nu_0.000000001/c_013.png
Simul